In [1]:
from sionna.rt import load_scene
import mitsuba as mi

import drjit as dr
import numpy as np
import torch

scene = load_scene(filename="cubes/cubes.xml")
# scene = load_scene(filename="minis/row4_col4.xml")

mi_scene = scene.mi_scene

In [2]:
shapes = mi_scene.shapes_dr()

# Will throw an error if there is a non-mesh shape
shapes = mi.MeshPtr(shapes)
print(shapes)

shape_ptrs = dr.reinterpret_array(mi.UInt32, shapes).numpy()
print(shape_ptrs)

face_normals = [
    s.face_normal(dr.arange(dr.cuda.UInt, s.face_count())).torch().T for s in shapes
]
face_normals = torch.vstack(face_normals)

# Find the average between the 3 vertices of a triangle mesh
# This will be the source for ray intersection testing
face_averages = []

# shape_ptr_to_offset = np.zeros(shape=shape_ptrs.shape[0]+1, dtype=int)
shape_ptr_to_offset = {}
offset_counter = 0
for i, s in enumerate(shapes):
    shape_ptr_to_offset[shape_ptrs[i]] = offset_counter
    offset_counter += s.primitive_count()

    indices = dr.arange(dr.cuda.UInt, s.face_count())
    vertex_indices = dr.ravel(s.face_indices(indices), order='F')
    vertices = s.vertex_position(vertex_indices)
    vertices = vertices.torch().T.reshape(-1, 3, 3)
    face_averages.append(vertices)

face_averages = torch.vstack(face_averages)
face_averages = face_averages.sum(1) / 3

face_averages = mi.Point3f(face_averages.T)
face_normals = mi.Vector3f(face_normals.T)
# print(face_averages)
# print(face_normals)

print(shape_ptr_to_offset)


[PLYMesh[
  name = "Cube_002.ply",
  bbox = BoundingBox3f[
    min = [-1, 3.58579, -1.41421],
    max = [1, 6.41421, 1.41421]
  ],
  vertex_count = 14,
  vertices = [280 B of vertex data],
  face_count = 12,
  faces = [144 B of face data],
  face_normals = 1
], PLYMesh[
  name = "Cube_001.ply",
  bbox = BoundingBox3f[
    min = [-6.65918, -1.56251, 1.67107],
    max = [-3.34082, 1.56251, 4.32893]
  ],
  vertex_count = 14,
  vertices = [280 B of vertex data],
  face_count = 12,
  faces = [144 B of face data],
  face_normals = 1
], PLYMesh[
  name = "Cube.ply",
  bbox = BoundingBox3f[
    min = [-1, -1, -1],
    max = [1, 1, 1]
  ],
  vertex_count = 14,
  vertices = [280 B of vertex data],
  face_count = 12,
  faces = [144 B of face data],
  face_normals = 1
]]
[2 3 1]
{2: 0, 3: 12, 1: 24}


In [3]:
# Generate rays to be shot from each surface in 180 cone
# Currently generates 360 then filters based on dot product

from sionna.rt.utils import fibonacci_lattice, spawn_ray_from_sources
ray_per_source = 10000

sources = face_averages + (face_normals * 0.0001)
rays = spawn_ray_from_sources(fibonacci_lattice, ray_per_source, sources) 
# print(rays)

norms_expanded = dr.repeat(face_normals, ray_per_source)
# print(norms_expanded)
# shouldn't this be positive dot prod???
active = dr.dot(rays.d, norms_expanded) > 0
# print(active)

In [4]:
import numpy as np

valid = np.array([True, True, False, True])
print(valid)
indices = np.nonzero(valid)[0]
print(indices)
add_mask = np.array([False, True, True])
print(add_mask)
set_false = indices[~add_mask]
valid[set_false] = False
print(valid)

[ True  True False  True]
[0 1 3]
[False  True  True]
[False  True False  True]


In [5]:
interactions = mi_scene.ray_intersect_preliminary(rays, active=active)
# print(interactions)
full_int = interactions.compute_surface_interaction(rays, active=active)
# print(full_int.p)

count = 0
for s in mi_scene.shapes():
    count += s.primitive_count()

valid = full_int.is_valid().numpy()
interaction_normals = full_int.n.numpy().T[valid]

# make sure the dot product is negative
# outgoing rays dirs
# maybe keep this in dr.jit
ray_dir_np = rays.d.numpy().T[valid]
print(ray_dir_np)
ray_dir_dot_interaction_normal_np = np.sum(ray_dir_np * interaction_normals, axis=1)
valid_hits_exterior = ray_dir_dot_interaction_normal_np < 0

indices = np.nonzero(valid)[0]
print(indices)

invalid = indices[~valid_hits_exterior]
valid[invalid] = False

# a=dr.select(interactions.is_valid() & (interactions.t > 1.5), interactions.prim_index, dr.cuda.UInt(count)).numpy()
# a=dr.select(full_int.is_valid(), full_int.prim_index, dr.cuda.UInt(count)).numpy()
# interaction_shapes=dr.select(full_int.is_valid(), full_int.shape, dr.cuda.UInt32(36000)).numpy()

indices = np.nonzero(valid)[0]
interaction_shapes = full_int.shape.numpy()[valid]
prim_indices = full_int.prim_index.numpy()[valid]
# interaction_shapes[0].id()
print(interaction_normals)

[[-0.70478445 -0.4767427   0.52535254]
 [-0.6662583  -0.53591406  0.5185518 ]
 [-0.62269825 -0.59191006  0.5117512 ]
 ...
 [ 0.05902368  0.9666195  -0.24932492]
 [-0.14980091  0.9560823  -0.25192523]
 [-0.02094843  0.9664165  -0.25612557]]
[  2373   2407   2441 ... 356246 356259 356280]
[[ 0.7600493   0.49367893 -0.4226183 ]
 [ 0.7600493   0.49367893 -0.4226183 ]
 [ 0.7600493   0.49367893 -0.4226183 ]
 ...
 [-0.         -0.70710677 -0.7071067 ]
 [-0.         -0.70710677 -0.7071067 ]
 [-0.         -0.70710677 -0.7071067 ]]


In [6]:
# print("shapes:", mi_scene.shapes())
# print(full_int.shape_index)
# # wtf=dr.reinterpret_array(mi.UInt32, full_int.shape)
# # wtf2 = dr.reinterpret_array(mi.ShapePtr, wtf)
# # # print(wtf)
# # for w in wtf2:
# #     print(w.id())
# print(wtf2[3598].has_mesh_attributes())
# np.unique(wtf)

In [7]:
# full_valid = full_int.is_valid().numpy()
# print(full_valid.shape)
# hit_points = full_int.p.numpy().T[full_valid]

# origins = rays.o.numpy().T[full_valid]
# print(origins.shape)

# scene.preview()

# # This cell is lowkey pretty illegal but here's to being easier to implement
# ## THIS REQUIRES persist=True in call self._add_child(mesh, pmin, pmax, persist=True) in _plot_lines (preview.py)


# colors = np.array([1.0, 0.0, 0.0], dtype=np.float32)
# colors = np.tile(colors, (origins.shape[0], 1))
# # print(colors)
# # print(colors.dtype)
# width = np.array(1.0)
# scene._preview_widget._plot_lines(origins, hit_points, colors, width)

In [8]:
# dir(full_int.prim_index)
# collision_shape_i = full_int.prim_index.numpy()[full_valid]

# np.unique(collision_shape_i)

In [9]:
scene.preview()
None

In [10]:
sets = [set() for _ in range(count)]

for i, shape_i, prim_i in zip(indices, interaction_shapes, prim_indices):
    # print(i, shape_i, prim_i)
    # shape_i -= 1
    # shape_id = mi_scene.shapes()[shape_i].id()
    offset = shape_ptr_to_offset[shape_i]
    sets[i // ray_per_source].add(offset + prim_i)

# for i in range(len(a)):
#     vis = a[i]
#     if vis < count:
#         cur_shape = interaction_shapes[i] - 1
#         cur_shape = mi_scene.shapes()[cur_shape].id()
#         offset = shape_offset[cur_shape]
#         # offset = mi_scene.shapes[collision_shape_i].id()
#         sets[i // ray_per_source].add(vis + offset)

sets
for i, s in enumerate(face_averages.numpy().T):
    print(s, sets[i])

[-0.33333334  4.0571914   0.47140452] {34, 35, 14, 15}
[0.33333334 4.528596   0.94280905] {34, 35, 14, 15}
[1.         5.         0.47140452] set()
[ 1.          5.         -0.47140452] set()
[ 0.33333334  4.0571914  -0.47140452] {34, 35, 14, 15}
[-0.33333334  4.528596   -0.94280905] {34, 35, 14, 15, 19}
[-0.33333334  5.942809   -0.47140452] set()
[ 0.33333334  5.4714046  -0.94280905] set()
[-0.33333334  5.4714046   0.94280905] set()
[0.33333334 5.942809   0.47140452] set()
[-1.        4.528596  0.      ] {18, 19, 14, 15}
[-1.         5.4714046  0.       ] {18, 19, 14, 15}
[-4.8267746 -1.0799167  2.8387704] {24, 25, 27, 28, 29}
[-4.083797  -0.5973266  3.1612298] {24, 25, 28, 29}
[-3.9402404   0.29087383  2.8794847 ] {0, 1, 4, 5, 10, 11, 24, 25, 28, 29}
[-4.5396614  0.6964841  2.2752793] {0, 1, 4, 5, 10, 11, 24, 25, 28, 29}
[-4.9194956  -0.34518716  1.9528196 ] {5, 10, 11, 28, 29}
[-5.7893386  -0.11522536  2.2345653 ] {10, 11, 28, 29}
[-5.6799254   0.75079745  3.4429755 ] {0, 1, 10, 11}

In [11]:
face_averages


[[-0.333333, 4.05719, 0.471405],
 [0.333333, 4.5286, 0.942809],
 [1, 5, 0.471405],
 .. 30 skipped ..,
 [1, -0.333333, -0.333333],
 [-0.333333, 1, 0.333333],
 [0.333333, 1, -0.333333]]

In [12]:
sizes = [len(s) for s in sets]
face_averages_np = face_averages.numpy().T
starts = np.repeat(face_averages_np, repeats=sizes, axis=0)
print(starts.shape)

ends_indices = []
for s in sets:
    for shape in s:
        ends_indices.append(shape)

ends_indices = np.array(ends_indices)
print(ends_indices)
ends = face_averages_np[ends_indices]
ends.shape

print(ends)

(115, 3)
[34 35 14 15 34 35 14 15 34 35 14 15 34 35 14 15 19 18 19 14 15 18 19 14
 15 24 25 27 28 29 24 25 28 29  0  1  4  5 10 11 24 25 28 29  0  1  4  5
 10 11 24 25 28 29  5 10 11 28 29 10 11 28 29  0  1 10 11  0  1 10 11  0
  1  1  0  1 12 13 14 15  0  1 12 13 14 15 12 12 12 13 14 15 16 12 13 14
 15 16 17  4  5  4  5  0  1  4  5 14 15  0  1  4  5 14 15]
[[-0.33333334  1.          0.33333334]
 [ 0.33333334  1.         -0.33333334]
 [-3.9402404   0.29087383  2.8794847 ]
 [-4.5396614   0.6964841   2.2752793 ]
 [-0.33333334  1.          0.33333334]
 [ 0.33333334  1.         -0.33333334]
 [-3.9402404   0.29087383  2.8794847 ]
 [-4.5396614   0.6964841   2.2752793 ]
 [-0.33333334  1.          0.33333334]
 [ 0.33333334  1.         -0.33333334]
 [-3.9402404   0.29087383  2.8794847 ]
 [-4.5396614   0.6964841   2.2752793 ]
 [-0.33333334  1.          0.33333334]
 [ 0.33333334  1.         -0.33333334]
 [-3.9402404   0.29087383  2.8794847 ]
 [-4.5396614   0.6964841   2.2752793 ]
 [-5.409504    0

In [13]:
scene.preview()

# This cell is lowkey pretty illegal but here's to being easier to implement
## THIS REQUIRES persist=True in call self._add_child(mesh, pmin, pmax, persist=True) in _plot_lines (preview.py)

# starts = np.array([[0,0,0],
#                    [1, 1, 1]])
# ends_indices = np.array([[5,5,5],
#                  [-100, -100, 0]])
colors = np.array([1.0, 0.0, 0.0], dtype=np.float32)
colors = np.tile(colors, (starts.shape[0], 1))
# print(colors)
# print(colors.dtype)
width = np.array(1.0)
scene._preview_widget._plot_lines(starts, ends, colors, width)